In [1]:
from ltlcross_runner import LtlcrossRunner
from IPython.display import display, display_markdown
import pandas as pd

If `rerun` is set to `False` the script uses the precomputed values. If set to `True`, all values are recomputed
(takes more time)

In [2]:
### Tools' setting ###
# PATHS
make_tgba   = 'ltl2tgba --deterministic -f %f' 
owl_version = '18.06'
owl_bin     = 'owl-'+owl_version+'/bin/'
# Tools
owl         = owl_bin+'ltl2ldgba -i %f -n'
nba2ldba    = make_tgba + ' --ba | '+owl_bin+'nba2ldba'
seminator   = make_tgba + ' | ./seminator'
cy          = make_tgba + ' | ./seminator --via-tba'
old         = make_tgba + ' | seminator'
# Optimizations
weak        = ' --powerset-for-weak'
entry       = ' --cut-on-SCC-entry'
levels      = ' --skip-levels'
avoid       = ' --bscc-avoid'
power       = ' --powerset-on-cut'
simpl_in    = ' --simplify-input'

# Options
cut         = ' --cd'
autfilt     = ' | autfilt'
new_autfilt = ' | ../spot_dev/bin/autfilt'
simp_sd     = ' --small --tgba'
simp_cd     = ' -x simul=1,ba-simul=1 --small --tgba'
nos         = ' -s0' # disables Spot's simplifications used in Seminator
end         = ' > %O' # saves result to file

def cd(tool):
    if tool.startswith('owl'):
        return tool
    return tool + cut

def sd(tool):
    if tool.startswith('owl'):
        return tool + ' -n'
    return tool

def simpl(tool, level, simpl):
    if (not tool.startswith('owl')) and level == 'no':
        tool += nos
    if level == 'no':
        return tool
    if level == 'yes':
        return tool + autfilt + simpl
    return tool + new_autfilt + simpl

d = {
    'old'          : old,
    'ltl2ldba'     : owl,
    'entry'        : seminator + entry,
    'e+simpl'      : seminator + entry + simpl_in,
    'es+weak'      : seminator + entry + simpl_in + weak,
    'es+power'     : seminator + entry + simpl_in + power,
    'esw+avoid'    : seminator + entry + simpl_in + weak + avoid,
    'esw+levels'   : seminator + entry + simpl_in + weak + levels,
    'eswal'        : seminator + entry + simpl_in + weak + avoid + levels,
    'eswpal'       : seminator + entry + simpl_in + weak + avoid + levels + power,
    
}

### Ltlcross runner configuration ###
tools = {}
    
for (name, cmd) in d.items():
    for level in ['no','yes','new']:
        tools[f'cd.{name}.{level}'] = simpl(cd(cmd), level, simp_cd) + end
        tools[f'sd.{name}.{level}'] = simpl(sd(cmd), level, simp_sd) + end

sd_tools = [t for t in tools.keys() if t.startswith('sd')]
cd_tools = [t for t in tools.keys() if t.startswith('cd')]
    
### Numbers to measure ###
cols = ['states','edges','transitions','nondet_states','exit_status','time','acc']
tool_order = list(d.keys())

In [3]:
rerun = True

In [4]:
runners = {}
for source in ('literature','random'):
    for t in ['nd']:
        name = '{}_{}'.format(source,t)
        runners[name] = \
            LtlcrossRunner(tools,\
                    res_filename='data/{}.csv'.format(name),\
                    formula_files=['formulae/{}.ltl'.format(name)],\
                    cols=cols)
        if rerun:
            runners[name].run_ltlcross(timeout='120',check=False)
        runners[name].parse_results()

In [5]:
def get_counts(runner):
    v = r.values.states.copy()
    v.columns = pd.MultiIndex.from_tuples([tuple(c.split('.')) for c in v.columns])
    return v.cd.dropna().count()[0], v.sd.dropna().count()[0]

## Cummulative results

Gather the data into one big DataFrame

In [6]:
data = pd.DataFrame()
for (name,r) in runners.items():
    # Divide into sd_tools & cd_tools to treat timeouts separately.
    tmp_cd = pd.DataFrame(r.cummulative(tools=cd_tools),columns=[name])
    tmp_sd = pd.DataFrame(r.cummulative(tools=sd_tools),columns=[name])
    data = data.append(tmp_cd.append(tmp_sd).transpose())
data

tool,cd.old.no,cd.old.yes,cd.old.new,cd.ltl2ldba.no,cd.ltl2ldba.yes,cd.ltl2ldba.new,cd.entry.no,cd.entry.yes,cd.entry.new,cd.e+simpl.no,...,sd.esw+avoid.new,sd.esw+levels.no,sd.esw+levels.yes,sd.esw+levels.new,sd.eswal.no,sd.eswal.yes,sd.eswal.new,sd.eswpal.no,sd.eswpal.yes,sd.eswpal.new
literature_nd,620,395,391,453,354,354,619,380,380,615,...,312,433,316,316,412,309,309,381,291,291
random_nd,2014,1131,1120,1085,833,833,1967,953,953,1906,...,686,1047,711,711,923,656,656,880,660,660


Get the number of formulas such that all tools finished within timeout for each category.

In [7]:
counts = pd.DataFrame(index=pd.Index([],name='source'),columns=['cd..n','sd..n'])
for (name,r) in runners.items():
    cd_c, sd_c = get_counts(r)
    counts = counts.append(pd.DataFrame({'cd..n':cd_c,'sd..n':sd_c},index=[name]))
counts

,cd..n,sd..n
literature_nd,23,23
random_nd,100,100


Converts the DataFrame to use MultiIndices for rows and columns

In [8]:
from pandas.api.types import CategoricalDtype
automata_type = CategoricalDtype(categories=['det','cd','sd','nd'], ordered=True)
source_type = CategoricalDtype(categories=['random', 'literature', 'rand. (T/O)', 'lit. (T/O)'], ordered=True)

In [9]:
def make_hierarchical(data):
    # Split index into origin & ltl2tgba's output type
    df = data.copy()
    df.index.name = 'origin'
    df.reset_index(inplace=True)
    df['type'] = df.origin.apply(lambda x: x.split('_')[1]).astype(automata_type)
    df['origin'] = df.origin.apply(lambda x: x.split('_')[0]).astype(source_type)
    df = df.set_index(['origin','type']).sort_index()
    # Split columns in tool & reductions
    df.columns = pd.MultiIndex.from_tuples([tuple(c.split('.')) for c in df.columns])
    return df

In [10]:
tmp = make_hierarchical(data.join(counts)).sort_index(axis=1)
tmp

cd                                                           \
                     e+simpl            entry            es+power               
                   n     new    no  yes   new    no  yes      new    no   yes   
origin     type                                                                 
random     nd    100     951  1906  951   953  1967  953     1027  1865  1027   
literature nd     23     380   615  380   380   619  380      350   584   350   

                ...     sd                                                  \
                ...  eswal eswpal           ltl2ldba             old         
                ...    yes    new   no  yes      new    no  yes  new    no   
origin     type ...                                                          
random     nd   ...    656    660  880  660      750  1088  750  896  1211   
literature nd   ...    309    291  381  291      327   455  327  348   444   

                      
                      
                 yes  
origin     type       
random     nd    919  
literature nd    352  

[2 rows x 62 columns]

In [11]:
tmp.cd

e+simpl            entry            es+power              \
                   n     new    no  yes   new    no  yes      new    no   yes   
origin     type                                                                 
random     nd    100     951  1906  951   953  1967  953     1027  1865  1027   
literature nd     23     380   615  380   380   619  380      350   584   350   

                 ...  eswal eswpal           ltl2ldba              old        \
                 ...    yes    new   no  yes      new    no  yes   new    no   
origin     type  ...                                                           
random     nd    ...    711    718  982  718      833  1085  833  1120  2014   
literature nd    ...    366    337  468  337      354   453  354   391   620   

                       
                  yes  
origin     type        
random     nd    1131  
literature nd     395  

[2 rows x 31 columns]

In [12]:
tmp.cd.swaplevel(axis=1).new

,,e+simpl,entry,es+power,es+weak,esw+avoid,esw+levels,eswal,eswpal,ltl2ldba,old
origin,type,,,,,,,,,,
random,nd,951,953,1027,951,739,922,711,718,833,1120
literature,nd,380,380,350,380,370,376,366,337,354,391


In [18]:
tmp.cd.swaplevel(axis=1).no

,,e+simpl,entry,es+power,es+weak,esw+avoid,esw+levels,eswal,eswpal,ltl2ldba,old
origin,type,,,,,,,,,,
random,nd,1906,1967,1865,1900,1052,1869,1025,982,1085,2014
literature,nd,615,619,584,614,502,611,499,468,453,620


In [19]:
tmp.sd.swaplevel(axis=1).new

,,e+simpl,entry,es+power,es+weak,esw+avoid,esw+levels,eswal,eswpal,ltl2ldba,old
origin,type,,,,,,,,,,
random,nd,741,741,749,741,686,711,656,660,750,896
literature,nd,319,319,300,319,312,316,309,291,327,348


In [20]:
tmp.sd.swaplevel(axis=1).no

,,e+simpl,entry,es+power,es+weak,esw+avoid,esw+levels,eswal,eswpal,ltl2ldba,old
origin,type,,,,,,,,,,
random,nd,1073,1159,1036,1069,944,1047,923,880,1088,1211
literature,nd,437,441,406,436,415,433,412,381,455,444


In [13]:
tmp.cd.eswpal

,,new,no,yes
origin,type,,,
random,nd,718,982,718
literature,nd,337,468,337


In [14]:
tmp.sd.eswpal

,,new,no,yes
origin,type,,,
random,nd,660,880,660
literature,nd,291,381,291


## Timeouts

In [21]:
TO_data = pd.DataFrame()
for (name,r) in runners.items():
    TO_data = TO_data.append(pd.DataFrame(r.compute_timeouts(),columns=[name]).transpose())
TO = make_hierarchical(TO_data).replace(0,pd.NaT).dropna(how='all').fillna(0)
TO

Empty DataFrame
Columns: [(cd, e+simpl, new), (cd, e+simpl, no), (cd, e+simpl, yes), (cd, entry, new), (cd, entry, no), (cd, entry, yes), (cd, es+power, new), (cd, es+power, no), (cd, es+power, yes), (cd, es+weak, new), (cd, es+weak, no), (cd, es+weak, yes), (cd, esw+avoid, new), (cd, esw+avoid, no), (cd, esw+avoid, yes), (cd, esw+levels, new), (cd, esw+levels, no), (cd, esw+levels, yes), (cd, eswal, new), (cd, eswal, no), (cd, eswal, yes), (cd, eswpal, new), (cd, eswpal, no), (cd, eswpal, yes), (cd, ltl2ldba, new), (cd, ltl2ldba, no), (cd, ltl2ldba, yes), (cd, old, new), (cd, old, no), (cd, old, yes), (sd, e+simpl, new), (sd, e+simpl, no), (sd, e+simpl, yes), (sd, entry, new), (sd, entry, no), (sd, entry, yes), (sd, es+power, new), (sd, es+power, no), (sd, es+power, yes), (sd, es+weak, new), (sd, es+weak, no), (sd, es+weak, yes), (sd, esw+avoid, new), (sd, esw+avoid, no), (sd, esw+avoid, yes), (sd, esw+levels, new), (sd, esw+levels, no), (sd, esw+levels, yes), (sd, eswal, new), (sd, eswal, no), (sd, eswal, yes), (sd, eswpal, new), (sd, eswpal, no), (sd, eswpal, yes), (sd, ltl2ldba, new), (sd, ltl2ldba, no), (sd, ltl2ldba, yes), (sd, old, new), (sd, old, no), (sd, old, yes)]
Index: []

[0 rows x 60 columns]

# Final results
We use the `display` function to propagate the results to notebooks that just run this one.

In [28]:
display_markdown('''### Comparison of tools producing cut-deterministic automata
All tools finished within the one-minute time limit.

#### Without simplifications
''',raw=True)

### Comparison of tools producing cut-deterministic automata
All tools finished within the one-minute time limit.

#### Without simplifications


In [29]:
display(tmp.cd.swaplevel(axis=1).no)

,,e+simpl,entry,es+power,es+weak,esw+avoid,esw+levels,eswal,eswpal,ltl2ldba,old
origin,type,,,,,,,,,,
random,nd,1906,1967,1865,1900,1052,1869,1025,982,1085,2014
literature,nd,615,619,584,614,502,611,499,468,453,620


In [31]:
display_markdown('#### With simplifications', raw=True)

#### With simplifications

In [32]:
display(tmp.cd.swaplevel(axis=1).yes)

,,e+simpl,entry,es+power,es+weak,esw+avoid,esw+levels,eswal,eswpal,ltl2ldba,old
origin,type,,,,,,,,,,
random,nd,951,953,1027,951,739,922,711,718,833,1131
literature,nd,380,380,350,380,370,376,366,337,354,395


In [33]:
display_markdown('''### Comparison of tools producing semi-deterministic automata
All tools finished within the one-minute time limit.

#### Without simplifications
''',raw=True)

### Comparison of tools producing semi-deterministic automata
All tools finished within the one-minute time limit.

#### Without simplifications


In [35]:
display(tmp.sd.swaplevel(axis=1).no)

,,e+simpl,entry,es+power,es+weak,esw+avoid,esw+levels,eswal,eswpal,ltl2ldba,old
origin,type,,,,,,,,,,
random,nd,1073,1159,1036,1069,944,1047,923,880,1088,1211
literature,nd,437,441,406,436,415,433,412,381,455,444


In [36]:
display_markdown('#### With simplifications', raw=True)

#### With simplifications

In [37]:
display(tmp.sd.swaplevel(axis=1).yes)

,,e+simpl,entry,es+power,es+weak,esw+avoid,esw+levels,eswal,eswpal,ltl2ldba,old
origin,type,,,,,,,,,,
random,nd,741,741,755,741,686,711,656,660,750,919
literature,nd,319,319,300,319,312,316,309,291,327,352


In [38]:
todo = runners['random_nd'].smaller_than('cd.ltl2ldba.new','cd.eswpal.new')
todo

,tool,cd.eswpal.new,cd.ltl2ldba.new
form_id,formula,,
1,G(Fa U X(b & Fc)),14,6
4,G((((a & b) | (!a & !b)) & (GF!b U !c)) | (((!a & b) | (a & !b)) & (FGb R c))),11,9
7,FG(Fa U (Fb R c)),32,4
10,G(a U Xb),3,2
15,GF((a & Gb) R c),6,5
17,GFa | G(Gb | Xa),5,4
19,FG((F((a & G!b) | (!a & Fb)) & (!b | F(b R c))) | (b & G((!b U !c) & ((!a & G!b) | (a & Fb))))),14,6
20,G(a | X(GFa | (a M b))) R !b,10,3
29,X(a U XG(b U (c | X(c M d)))),5,4


In [39]:
list(todo.index.droplevel().values)

['G(Fa U X(b & Fc))',
 'G((((a & b) | (!a & !b)) & (GF!b U !c)) | (((!a & b) | (a & !b)) & (FGb R c)))',
 'FG(Fa U (Fb R c))',
 'G(a U Xb)',
 'GF((a & Gb) R c)',
 'GFa | G(Gb | Xa)',
 'FG((F((a & G!b) | (!a & Fb)) & (!b | F(b R c))) | (b & G((!b U !c) & ((!a & G!b) | (a & Fb)))))',
 'G(a | X(GFa | (a M b))) R !b',
 'X(a U XG(b U (c | X(c M d))))',
 'GF((Ga & F((b & G!c) | (!b & Fc))) | (F!a & G((b & Fc) | (!b & G!c))))',
 'GF((a & F(!b | Xc)) | (!a & G(b & X!c)))',
 'XG(a | Fc | Xb)',
 'FG(a | XF(a & XFb))',
 'FG((a & (!b | c | XFb | Gd)) | (!a & b & !c & XG!b & F!d))',
 'XG(a & F((!b & F!c) | (b & Gc)))',
 'GF(a | G(Fb & F!a))',
 'X(a & GF(b & X(b W c)))',
 'GF((a & X(b & (FGc U Xc))) | ((!a | X!b) & X(GF!c R X!c)))',
 'FG(F!a & (b U Xa))']